In [ ]:
import socket
import aiohttp
import asyncio


class Flashlight:
    commands = {"on": 'turn_on', "off": 'turn_off', "color": 'change_color'}
    colors = {1: "white", 2: "red", 3: "blue", 4: "green"}

    def __init__(self, status=0, color=1):
        self.status = status
        self.color = self.colors[color]

    async def command_accept(self, command, metadata=1):

        if self.command in self.commands:
            method_name = self.commands[command]
            method = getattr(self, method_name)
            method(metadata)

    def turn_on(self, status):
        if status == 0:
            status = 1
            print("Фонарь включен")
        else:
            print("Фонарь уже включен")
        return status

    def turn_off(self, status):
        if status == 1:
            status = 0
            print("Фонарь выключен")
        else:
            print("Фонарь не включен")
        return status

    def change_color(self, status):
        a = int(input("Выберите цвет "))
        self.color = a
        print(f"Выставлен {self.colors[a]} цвет") if status == 1 else print("Фонарь не включен")


async def processing_messages(message, flashlight):
    command = message["command"].lover()
    metadata = message.get("metadata")
    await flashlight.command_accept(command, metadata)


async def main():
    while True:
        host = input("input address ") or "127.0.0.1"
        port = int(input("input port ") or 9999)
        if not (0 < port < 65535):
            raise ValueError
        break

    flashlight = Flashlight()
    session = aiohttp.ClientSession()
    print(f'http://{host}:{port}')
    async with session.ws_connect(f'http://{host}:{port}') as ws:
        async for msg in ws:
            if msg.type == aiohttp.WSMsgType.TEXT:
                message = msg.json()
                await processing_messages(message, flashlight)
            elif msg.type == aiohttp.WSMsgType.ERROR:
                print("error")
                break



asyncio.run(main())